# License
    IPython notebook for simulating the linear wave equation with CUDA
    Copyright (C) 2015, 2018 Andre.Brodtkorb@ifi.uio.no

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [8]:
%matplotlib inline 

#Import packages we need
import numpy as np
from matplotlib import animation, rc, cm
from matplotlib import pyplot as plt

import pycuda.compiler as cuda_compiler
import pycuda.driver as cuda_driver
from pycuda.gpuarray import GPUArray

import IPythonMagic
from Timer import Timer

import pytest
from ipytest import run_pytest, clean_tests

In [9]:
# From IPythonMagic
%setup_logging
%cuda_context_handler context

Global logger already initialized!
Registering context in user workspace
Context already registered! Ignoring


In [10]:
class HeatEquation1D(object):

    """
    Explicit finite differences for 1D heat equation
    """
    
    def __init__(self, u0, kappa, dx, dt):
        self.u0 = u0.copy()
        self.u1 = np.empty_like(self.u0)
        self.kappa = kappa
        self.dx = dx
        self.dt = dt
        self.nx = self.u0.shape[0] - 2 
        
    def step(self):
        for i in range(1,self.nx+1):
            self.u1[i] = self.u0[i] + self.kappa*self.dt/self.dx**2 * (self.u0[i-1] - 2.0*self.u0[i] + self.u0[i+1])
    
        self.u1[0] = self.u1[1]
        self.u1[self.nx+1] = self.u1[self.nx]
        
        # Swap u0 with u1
        self.u0, self.u1 = self.u1, self.u0
        
    def download(self):
        return self.u0.copy()
    

In [22]:
class HeatEquation1DGPU(object):

    """
    Explicit finite differences for 1D heat equation
    """
    
    def __init__(self):
        pass
    
    def createKernel(self):
        src = """
        __global__ void heatEquation(float* u1, const float* u0, float kappa, float dx, float dt, int nx){
        
            // Skip the first ghost cell ==> +1
            int i = blockIdx.x*blockDim.x + threadIdx.x + 1;
        
            if (i>=1 && i <= nx) {
                u1[i] = u0[i] + kappa*dt/(dx*dx) * (u0[i-1] - 2.0f*u0[i] + u0[i+1]);
            }   
            
            if (i == 1) {
                u1[0] = u0[i] + kappa*dt/(dx*dx) * (u0[i-1] - 2.0f*u0[i] + u0[i+1]);
            }   
            
            if (i == nx) {
                u1[nx+1] = u0[i] + kappa*dt/(dx*dx) * (u0[i-1] - 2.0f*u0[i] + u0[i+1]);
            }   
        }
        
        __device__ int computePi(){
            return 22.0f/7.0f;
        }
        """       
        
        self.module = cuda_compiler.SourceModule(src, options=['--use_fast_math'])
        self.kernel = self.module.get_function("heatEquation");
        self.kernel.prepare("PPfffi")
    
    def initialize(self, u0, kappa, dx, dt):
        self.kappa = np.float32(kappa)
        self.dx = np.float32(dx)
        self.dt = np.float32(dt)
        self.nx = np.int32(u0.shape[0] - 2) 
       
        assert u0.dtype == np.float32, "u0 must be a float"
        self.u0_g = GPUArray(u0.shape,u0.dtype)
        self.u1_g = GPUArray(u0.shape,u0.dtype)
        self.u0_g = set(u0)
        
        self.createKernel()
        
        num_threads = 128
        self.block_size = (num_threads, 1, 1)
        self.grid_size = (int(np.ceil(self.u0_g.shape[0] /num_threads)), 1, 1)
        self.stream = cuda_driver.Stream()
        print(grid_size)
        print(block0_size)
        print(u0_g.shape)
        
        self.kernel.prepared_async_call(self.grid_size, self.block_size, self.stream, \
                                        self.u1_g.gpudata, self.u0_g.gpudata, \
                                        self.kappa, self.dx, self.dt, self.nx)
        
    def step(self):
        for i in range(1,self.nx+1):
            self.u1[i] = self.u0[i] + self.kappa*self.dt/self.dx**2 * (self.u0[i-1] - 2.0*self.u0[i] + self.u0[i+1])
    
        self.u1[0] = self.u1[1]
        self.u1[self.nx+1] = self.u1[self.nx]
        
        # Swap u0 with u1
        self.u0_g, self.u1_g = self.u1_g, self.u0_g
        
    def download(self):
        u0 = np.empty(self.u0_g.shape,self.u0_g.dtype)
        self.u0_g.get(u0)
        return u0
    

In [23]:
u0 = np.zeros(21,dtype=np.float32)
u0[10] = 1.0
kappa = 1.0
dx = 1.0 
dt = 0.4*dx**2/(2*kappa)

gpu_simulator = HeatEquation1DGPU()
gpu_simulator.initialize(u0, kappa, dx, dt)

gpu_simulator.step()
result = gpu_simulator.download()
print(result)

#for i in range(10):
#    simulator = HeatEquation1DGPU(u0, kappa, dx, dt)
#    simulator.step()
#    result = simulator.download()

#    fig = plt.figure()
#    plt.plot(result)

AttributeError: 'set' object has no attribute 'shape'

In [ ]:
clean_tests()

def test_HeatEquation1D():
    u0 = np.zeros(15)
    u0[3] = 1.0
    kappa = 1.0
    dx = 1.0 
    dt = 0.4*dx**2/(2*kappa)

    simulator = HeatEquation1D(u0, kappa, dx, dt)
    
    for i in range(10):
        simulator.step()
        result = simulator.download()
        print(result)
        assert np.sum(result[1:-1]) == pytest.approx(1.0)
        assert result[0] == result[1]
        assert result[u0.shape[0]-1] == result[u0.shape[0]-2] 
    
    #assert np.all(u0 == simulator.u0)
    #assert simulator.u0.shape == simulator.u1.shape
    #assert kappa == simulator.kappa
    #assert dx == simulator.dx
    #assert dt == simulator.dt
    
run_pytest(filename="14 HeatEquation1D-Fede.ipynb",pytest_options=["-vvv"])